In [4]:
# This code is modified from Opencv SVM tutorial
# https://docs.opencv.org/trunk/d3/d02/tutorial_py_svm_index.html

In [1]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
# Deskewing the image with its second order moments
def deskew(img):
    moments = cv2.moments(img)
    if abs(moments['mu02']) < 1e-2:
        # No deskewing is needed
        return img.copy()
    # Calculate the skew based on central moments
    skew = moments['mu11'] / moments['mu02']
    mat = np.float32([[1, skew, -0.5*size*skew], [0, 1, 0]])
    img = cv2.warpAffine(img, mat, (size, size), flags=cv2.WARP_INVERSE_MAP | cv2.INTER_LINEAR)
    
    return img

def hog(img):
    gx = cv2.Sobel(img, cv2.CV_32F, 1, 0)
    gy = cv2.Sobel(img, cv2.CV_32F, 0, 1)
    magnitude, angle = cv2.cartToPolar(gx, gy)
    # quantizing binvalues in (0 - 16)
    bins = np.int32(num_bins * angle / (2 * np.pi))
    bin_cells = bins[:10, :10], bins[10:, :10], bins[:10, 10:], bins[10:, 10:]
    mag_cells = magnitude[:10, :10], magnitude[10:, :10], magnitude[:10, 10:], magnitude[10:, 10:]
    hists = [np.bincount(b.ravel(), m.ravel(), num_bins) for b, m in zip(bin_cells, mag_cells)]
    hist = np.hstack(hists)
    
    return hist

In [8]:
size = 20
num_bins = 16

# Data preparation
img= cv2.imread('images/digits.png', 0)

if img is None:
    raise Exception("Please check the availability of the digits.png file!")

# Split the image to 5000 cells with each in size 20x20
cells = [np.hsplit(row, 100) for row in np.vsplit(gray, 50)]
train_cells = [i[:50] for i in cells]
test_cells = [i[50:] for i in cells]

(50, 50, 20, 20)

In [9]:
deskewed = [list(map(deskew, row)) for row in train_cells]
hog_data = [list(map(hog, row)) for row in deskewed]
train_data = np.float32(hog_data).reshape(-1, 64)
labels = np.repeat(np.arange(10), 250)[:, np.newaxis]

In [12]:
# Initialize the SVM model and train the data
svm = cv2.ml.SVM_create()
svm.setKernel(cv2.ml.SVM_LINEAR)
svm.setType(cv2.ml.SVM_C_SVC)
# Below two parameters are gained by trial and error
svm.setC(2.67)
svm.setGamma(5.383)

svm.train(train_data, cv2.ml.ROW_SAMPLE, labels)
svm.save('svm_data.dat')

In [14]:
# Test the SVM model
deskewed_test = [list(map(deskew, row)) for row in test_cells]
hog_test = [list(map(hog, row)) for row in deskewed_test]
test_data = np.float32(hog_test).reshape(-1, 64)
predictions = svm.predict(test_data)[1]

In [19]:
#mask = np.array(result==responses)
#correct = np.count_nonzero(mask)
#print(correct * 100.0/ result.size)

accuracy = (predictions == responses).mean()
print(accuracy)

0.938
